In [21]:
import matplotlib.pyplot as plt

# Load the selected data
selected_data = pd.read_csv('C:/Users/ksultana/Videos/Internship/selected_data.csv')


counts = selected_data['selection_type'].value_counts()

# Print the counts
print(counts)


# Calculate class weights based on the inverse of class counts
class_counts = selected_data['selection_type'].value_counts()
total_samples = len(selected_data)

class_weights = {0: total_samples / class_counts[0],
                 1: total_samples / class_counts[1],
                 2: total_samples / class_counts[2]}

# Normalize the weights so that the maximum weight is 10
max_weight = max(class_weights.values())
class_weights_normalized = {key: value / max_weight * 10 for key, value in class_weights.items()}

# Create the logistic regression model with class weights
model_weighted = LogisticRegression(solver='liblinear', class_weight=class_weights_normalized).fit(X_train, y_train)

# Evaluate the model
y_pred_weighted = model_weighted.predict(X_test)

# Print the updated classification report
print("Classification Report with Adjusted Class Weights:")
print(classification_report(y_test, y_pred_weighted))


0    517
2     20
1     19
Name: selection_type, dtype: int64
Classification Report with Adjusted Class Weights:
              precision    recall  f1-score   support

           0       0.94      0.96      0.95       157
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         6

    accuracy                           0.90       167
   macro avg       0.31      0.32      0.32       167
weighted avg       0.88      0.90      0.89       167



In [10]:
# Perform logistic regression on the training set using solver='liblinear'
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear').fit(X_train, y_train)

# Coefficients and intercept
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

Coefficients: [[-0.06789506  0.00809226  0.12118406  0.01483623 -0.05723382  0.00581175
  -0.00777832  0.39603517]
 [-0.36026219  0.00272353 -0.19596212 -0.01504033  0.09632335 -0.00161262
   0.04093398 -0.83090017]
 [ 0.08234535 -0.03476912 -0.08293095 -0.03043077  0.01733158 -0.02626551
  -0.04558308  0.1662237 ]]
Intercept: [ 0.97561383 -0.73759926 -0.24647045]


In [11]:
# Print the training and testing accuracy of the model
from sklearn.metrics import accuracy_score
train_accuracy = accuracy_score(y_train, model.predict(X_train))
test_accuracy = accuracy_score(y_test, model.predict(X_test))
print(f"Training Accuracy: {train_accuracy:.2f}")
print(f"Testing Accuracy: {test_accuracy:.2f}")

Training Accuracy: 0.93
Testing Accuracy: 0.94


In [12]:
# Print the confusion matrix
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, model.predict(X_test))
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[157   0   0]
 [  4   0   0]
 [  6   0   0]]


In [15]:
# Print the classification report
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report
class_report = classification_report(y_test, model.predict(X_test))
print("Classification Report:")
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       157
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         6

    accuracy                           0.94       167
   macro avg       0.31      0.33      0.32       167
weighted avg       0.88      0.94      0.91       167



In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load the selected data
selected_data = pd.read_csv('C:/Users/ksultana/Videos/Internship/selected_data.csv')

# Separate features (X) and target variable (y)
X = selected_data.drop('selection_type', axis=1)
y = selected_data['selection_type']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define class weights based on the inverse of class counts
class_counts = y_train.value_counts()
class_weights = dict(1 / class_counts)

# Create and fit the RandomForestClassifier with class weights
rf = RandomForestClassifier(class_weight=class_weights, random_state=42)
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf.predict(X_test)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       157
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         6

    accuracy                           0.94       167
   macro avg       0.31      0.33      0.32       167
weighted avg       0.88      0.94      0.91       167

